https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/gan.py
https://stackoverflow.com/questions/40994583/how-to-implement-tensorflows-next-batch-for-own-data

In [2]:
import xml.etree.ElementTree as ET
import csv


def get_labels_dict(data_path):
    labels_dict = {}
    with open(data_path + 'sci_labels.csv', 'r') as f:
        file = csv.reader(f)
        for row in file:
            labels_dict[row[0]] = row[1]
    return labels_dict


def get_features_labels(root, labels_dict):
    corpus = [] # each row is a string formed from all messages in a conversations
    labels = [] # each row is 0 or 1, corresponds to label for same row in corpus

    for conversation in root:
        string = " "
        for message in conversation:
            text = message.find('text').text
            if text is not None:
                string = string + "\r\n" + text 
        corpus.append(string)
        labels.append(int(labels_dict[conversation.get('id')]))
    return corpus, labels

In [3]:
train_data_path = '../../data/svm_training_data/'
training_xml = ET.parse(train_data_path + 'training_data.xml')
train_root = training_xml.getroot()

test_data_path = '../../data/svm_test_data/'
test_data_src = '../../data/pan12-sexual-predator-identification-test-corpus-2012-05-21/'
test_xml = ET.parse(test_data_src + 'pan12-sexual-predator-identification-test-corpus-2012-05-17.xml')
test_root = test_xml.getroot()

train_corpus, train_labels = get_features_labels(train_root, get_labels_dict(train_data_path))
test_corpus, test_labels = get_features_labels(test_root, get_labels_dict(test_data_path))

train_corpus_norm = []
train_corpus_susp = []
train_labels_norm = []
train_labels_susp = []
for index in range(len(train_corpus)):
    if train_labels[index] == 1:
        train_corpus_susp.append(train_corpus[index])
        train_labels_susp.append(train_labels[index])
    else:
        train_corpus_norm.append(train_corpus[index])
        train_labels_norm.append(train_labels[index])

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MaxAbsScaler
import scipy
# from sklearn.model_selection import train_test_split
import numpy as np

vectorizer = TfidfVectorizer()
_ = vectorizer.fit_transform(train_corpus)
X_train_norm = vectorizer.transform(train_corpus_norm)
X_train_susp = vectorizer.transform(train_corpus_susp)
X_test = vectorizer.transform(test_corpus)

X_train_norm = scipy.sparse.csr_matrix(X_train_norm, dtype=np.float32)
y_train_norm = np.array(train_labels_norm)
X_train_susp = scipy.sparse.csr_matrix(X_train_susp, dtype=np.float32)
y_train_susp = np.array(train_labels_susp)
X_test = scipy.sparse.csr_matrix(X_test, dtype=np.float32)
y_test = np.array(test_labels)

print(np.min(X_train_norm[:][0]))
print(np.max(X_train_norm[:][0]))

scaler = MaxAbsScaler()
X_train_norm = scaler.fit_transform(X_train_norm)
X_train_susp = scaler.fit_transform(X_train_susp)
X_test = scaler.fit_transform(X_test)

print(np.min(X_train_norm[:][0]))
print(np.max(X_train_norm[:][0]))

# X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=87)
# print("Train data shape:{}\r\nTest data shape:{}".format(X_train.shape, X_test.shape))

0.0
0.44709805
0.0
1.0


In [5]:
# print(X_train_norm[:][0])
print(X_train_norm[0])

  (0, 119398)	0.081819706
  (0, 119332)	0.14805734
  (0, 119276)	0.4289901
  (0, 118973)	0.041549932
  (0, 118941)	0.07015078
  (0, 118779)	0.4489788
  (0, 118505)	0.09728563
  (0, 117862)	0.047014344
  (0, 116778)	0.30102143
  (0, 116483)	0.08699928
  (0, 115839)	0.36217767
  (0, 115808)	0.094457544
  (0, 115667)	0.0963027
  (0, 115510)	0.047327537
  (0, 115439)	0.21800916
  (0, 115238)	0.22017342
  (0, 115186)	0.18904479
  (0, 115042)	0.18243727
  (0, 114443)	0.17148164
  (0, 114358)	0.15560317
  (0, 113295)	0.40773752
  (0, 112762)	0.077801585
  (0, 112462)	0.15320362
  (0, 111681)	0.10570731
  (0, 111189)	0.09465508
  :	:
  (0, 24726)	0.20149395
  (0, 23331)	0.26914763
  (0, 22067)	0.12671371
  (0, 21787)	0.070174195
  (0, 21531)	0.2797445
  (0, 21493)	0.12715293
  (0, 21457)	0.35625395
  (0, 21160)	0.17045741
  (0, 20994)	0.30733147
  (0, 20764)	0.13206111
  (0, 19740)	0.09832926
  (0, 18206)	0.28438988
  (0, 17402)	0.04014927
  (0, 16970)	0.5268748
  (0, 16725)	0.11428115
  (0, 1

In [6]:
""" Generative Adversarial Networks (GAN).
Using generative adversarial networks (GAN) to generate digit images from a
noise distribution.
References:
    - Generative adversarial nets. I Goodfellow, J Pouget-Abadie, M Mirza,
    B Xu, D Warde-Farley, S Ozair, Y. Bengio. Advances in neural information
    processing systems, 2672-2680.
    - Understanding the difficulty of training deep feedforward neural networks.
    X Glorot, Y Bengio. Aistats 9, 249-256
Links:
    - [GAN Paper](https://arxiv.org/pdf/1406.2661.pdf).
    - [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
    - [Xavier Glorot Init](www.cs.cmu.edu/~bhiksha/courses/deeplearning/Fall.../AISTATS2010_Glorot.pdf).
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import division, print_function, absolute_import

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Import MNIST data
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Params
num_steps = 10
batch_size = 100 #128
learning_rate = 0.0002

# Network Params
image_dim = X_train_norm.shape[1] #784 # 28*28 pixels
gen_hidden_dim = 10 #256
disc_hidden_dim = 10 #256
noise_dim = X_train_norm.shape[1] # 100 # Noise data points
print(X_train_norm.shape[1])

# A custom initialization (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

# Store layers weight & bias
weights = {
    'gen_hidden1': tf.Variable(glorot_init([noise_dim, gen_hidden_dim])),
    'gen_out': tf.Variable(glorot_init([gen_hidden_dim, image_dim])),
    'disc1_hidden1': tf.Variable(glorot_init([image_dim, disc_hidden_dim])),
    'disc1_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
    'disc2_hidden1': tf.Variable(glorot_init([image_dim, disc_hidden_dim])),
    'disc2_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
}
biases = {
    'gen_hidden1': tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out': tf.Variable(tf.zeros([image_dim])),
    'disc1_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc1_out': tf.Variable(tf.zeros([1])),
    'disc2_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc2_out': tf.Variable(tf.zeros([1])),
}


# Generator
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['gen_out'])
    out_layer = tf.add(out_layer, biases['gen_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer


# Discriminator
def discriminator_SCI(x): # is D in paper
    hidden_layer = tf.matmul(x, weights['disc1_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc1_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc1_out'])
    out_layer = tf.add(out_layer, biases['disc1_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

def discriminator_gvr(x): # is D prime in paper, discriminator_generated_vs_real
    hidden_layer = tf.matmul(x, weights['disc2_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc2_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc2_out'])
    out_layer = tf.add(out_layer, biases['disc2_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , data.shape[0])
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i].toarray() for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    data_shuffle = scipy.sparse.csr_matrix(data_shuffle)
    print(data_shuffle.shape)
    return data_shuffle, labels_shuffle
#     print(len(data_shuffle[0]))
#     return scipy.sparse.csr_matrix(data_shuffle), np.asarray(labels_shuffle)
#     return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

121394
Instructions for updating:
Colocations handled automatically by placer.


In [7]:
# Build Networks
# Network Inputs
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
# disc_input_normal = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_normal')
# disc_input_real_susp = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_real_susp')
# # disc_input_fake_susp = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_fake_susp')

# tf.float32
# https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428
# x_norm = tf.sparse_placeholder(tf.float64)
# x_susp = tf.sparse_placeholder(tf.float64)
x_norm, x_susp = tf.placeholder(tf.float32, shape=[None, image_dim]), tf.placeholder(tf.float32, shape=[None, image_dim])
dataset_norm = tf.data.Dataset.from_tensor_slices(x_norm).repeat().batch(batch_size)
dataset_susp = tf.data.Dataset.from_tensor_slices(x_susp).repeat().batch(batch_size)
# train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
# test_data = (np.array([[1,2]]), np.array([[0]]))
iter_norm = dataset_norm.make_initializable_iterator()
iter_susp = dataset_susp.make_initializable_iterator()
features_norm = iter_norm.get_next()
features_susp = iter_susp.get_next()

# Build Generator Network
gen_sample = generator(gen_input)

# Build 2 Discriminator Networks (one from noise input, one from generated samples)
# disc_SCI_normal = discriminator_SCI(disc_input_normal)
# disc_SCI_susp_real = discriminator_SCI(disc_input_real_susp)
# disc_SCI_susp_fake = discriminator_SCI(gen_sample)
# disc_gvr_real = discriminator_gvr(disc_input_real_susp)
# disc_gvr_fake = discriminator_gvr(gen_sample)

disc_SCI_normal = discriminator_SCI(features_norm)
disc_SCI_susp_real = discriminator_SCI(features_susp)
disc_SCI_susp_fake = discriminator_SCI(gen_sample)
disc_gvr_real = discriminator_gvr(features_susp)
disc_gvr_fake = discriminator_gvr(gen_sample)


# Build Loss
gen_loss = -tf.reduce_mean(tf.log(disc_SCI_susp_fake) + tf.log(disc_gvr_fake))
disc_SCI_loss = -tf.reduce_mean(tf.log(disc_SCI_normal) + tf.log(1. - disc_SCI_susp_real) + tf.log(1. - disc_SCI_susp_fake))
disc_gvr_loss = -tf.reduce_mean(tf.log(disc_gvr_real) + tf.log(1. - disc_gvr_fake))

# Build Optimizers
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Training Variables for each optimizer
# By default in TensorFlow, all variables are updated by each optimizer, so we
# need to precise for each one of them the specific variables to update.
# Generator Network Variables
gen_vars = [weights['gen_hidden1'], weights['gen_out'],
            biases['gen_hidden1'], biases['gen_out']]
# Discriminator Network Variables
disc_SCI_vars = [weights['disc1_hidden1'], weights['disc1_out'],
            biases['disc1_hidden1'], biases['disc1_out']]
disc_gvr_vars = [weights['disc2_hidden1'], weights['disc2_out'],
            biases['disc2_hidden1'], biases['disc2_out']]

# Create training operations
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc_SCI = optimizer_disc.minimize(disc_SCI_loss, var_list=disc_SCI_vars)
train_disc_gvr = optimizer_disc.minimize(disc_gvr_loss, var_list=disc_gvr_vars)

print(X_train_norm.shape)
print(X_train_susp.shape)

(13802, 121394)
(901, 121394)


In [9]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    sess.run([iter_norm.initializer, iter_susp.initializer], feed_dict={ x_norm: X_train_norm.todense(), x_susp: X_train_susp.todense()})
    
    for i in range(1, num_steps+1):
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
#         batch_x, _ = mnist.train.next_batch(batch_size)
#         batch_x_norm, _ = next_batch(batch_size, X_train_norm, y_train_norm)
#         batch_x_susp, _ = next_batch(batch_size, X_train_susp, y_train_susp)
#         print(batch_x_norm.shape)
#         print(batch_x_susp.shape)
        # Generate noise to feed to the generator
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        print()
        # Train
#         feed_dict = {disc_input: batch_x, gen_input: z}
        feed_dict = {gen_input: z}
        _, _, _, gl, dSCIl, dgvrl = sess.run([train_gen, train_disc_SCI, train_disc_gvr, gen_loss, disc_SCI_loss, disc_gvr_loss],
                                feed_dict=feed_dict)
#         if i % 1000 == 0 or i == 1:
        print('Step %i: Generator Loss: %f, Disc SCI Loss: %f, Disc gvr Loss: %f' % (i, gl, dSCIl, dgvrl))

    print("Finished Training")
    # test SCI
    sess.run(iter_norm.initializer, feed_dict={ x_norm: X_test.todense()})
    y_pred = sess.run(disc_SCI_normal)
    print(metrics.accuracy_score(y_test, y_pred))
#     # Generate images from noise, using the generator network.
#     f, a = plt.subplots(4, 10, figsize=(10, 4))
#     for i in range(10):
#         # Noise input.
#         z = np.random.uniform(-1., 1., size=[4, noise_dim])
#         g = sess.run([gen_sample], feed_dict={gen_input: z})
#         g = np.reshape(g, newshape=(4, 28, 28, 1))
#         # Reverse colours for better display
#         g = -1 * (g - 1)
#         for j in range(4):
#             # Generate image from noise. Extend to 3 channels for matplot figure.
#             img = np.reshape(np.repeat(g[j][:, :, np.newaxis], 3, axis=2),
#                              newshape=(28, 28, 3))
#             a[j][i].imshow(img)

#     f.show()
#     plt.draw()
#     plt.waitforbuttonpress()


Step 1: Generator Loss: 1.490234, Disc SCI Loss: 2.102442, Disc gvr Loss: 1.277664

Step 2: Generator Loss: inf, Disc SCI Loss: 1.383820, Disc gvr Loss: 0.695601

Step 3: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan

Step 4: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan

Step 5: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan

Step 6: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan

Step 7: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan

Step 8: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan

Step 9: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan

Step 10: Generator Loss: nan, Disc SCI Loss: nan, Disc gvr Loss: nan
Finished Training


MemoryError: 